In [ ]:
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import ee
import joblib
import geemap

from sklearn.utils import shuffle
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

from shapely.geometry import Point, Polygon



In [ ]:
# Authenticate and initialize the Earth Engine API
ee.Authenticate()
ee.Initialize(project='svevazanettieo1') #così apro il progetto esistente in GEE, non ne sto creando uno nuovo

In [ ]:
scaler = joblib.load("scaler.save")
clf = joblib.load("random_forest_model_train+val.pkl")

In [ ]:
# Function to convert Earth Engine FeatureCollection to Pandas DataFrame, eliminating the random column
def from_FeatureCollection_to_df(path):
    points = ee.FeatureCollection(path)
    points_ft = points.getInfo()['features']
    points_df = pd.json_normalize(points_ft)
    points_df = points_df.drop(columns=['properties.random'], errors='ignore')
    return points_df

testing_points_df_2018 = from_FeatureCollection_to_df('projects/svevazanettieo1/assets/Dubai/Dubai_testing_indexes')
testing_points_df_2024 = from_FeatureCollection_to_df('projects/svevazanettieo1/assets/Dubai/Dubai_testing_indexes_2024')

print('Testing set 2018:', testing_points_df_2018.shape)
print('Testing set 2024:', testing_points_df_2024.shape)

In [ ]:
# Extract feature columns (e.g., 'properties.B1', 'properties.B2', etc.)
feature_columns = [col for col in testing_points_df_2018.columns if col.startswith('properties.B') or col.startswith('properties.n')]
feature_columns

# Extract features and labels for testing
X_test_2018 = testing_points_df_2018[feature_columns]
y_test_2018 = testing_points_df_2018['properties.LC']

X_test_2024 = testing_points_df_2024[feature_columns]
y_test_2024 = testing_points_df_2024['properties.LC']

In [ ]:
X_test_2018 = scaler.transform(X_test_2018)
X_test_2024 = scaler.transform(X_test_2024)

# Testing 2018

In [ ]:
X_shuffled_test_2018, y_shuffled_test_2018, testing_points_df_2018 = shuffle(X_test_2018, y_test_2018, testing_points_df_2018, random_state=42)
y_shuffled_test_2018 = np.argmax(y_shuffled_test_2018, axis=1)

# Predicting using the model
y_pred_test_2018 = clf.predict(X_shuffled_test_2018)  # Get predictions from the model

In [ ]:
# Compute confusion matrix
cm_2018 = confusion_matrix(y_shuffled_test_2018, y_pred_test_2018)

# Plot
fig, ax = plt.subplots(figsize=(8, 6)) 
disp = ConfusionMatrixDisplay(confusion_matrix=cm_2018, display_labels=['vegetation','water','soil','urban'])
disp.plot(cmap="Blues", values_format="d", ax=ax, colorbar=False)  # use ax
ax.set_title("Confusion Matrix", fontsize=16)
plt.tight_layout()
plt.show()

In [ ]:
testing_points_df_2018['predicted'] = y_pred_test_2018

def make_geometry(row):
    if row['geometry.type'] == 'Point':
        return Point(row['geometry.coordinates'])
    elif row['geometry.type'] == 'Polygon':
        return Polygon(row['geometry.coordinates'])
    else:
        return None  # o gestisci altri tipi se ci sono

testing_points_df_2018['geometry'] = testing_points_df_2018.apply(make_geometry, axis=1)
gdf_2018 = gpd.GeoDataFrame(testing_points_df_2018, geometry='geometry')
gdf_2018.set_crs(epsg=4326, inplace=True)

In [ ]:
sentinel2018 = ee.Image('projects/svevazanettieo1/assets/Dubai/sentinel2018')
vis_params = {
    'bands': ['B4', 'B3', 'B2'],  # RGB
    'min': 0,
    'max': 0.3,
}

# Select only the necessary columns for visualization
test_points_RF = geemap.geopandas_to_ee(gdf_2018[['predicted', 'geometry']])

# Convert FeatureCollection to Image
test_image = test_points_RF.reduceToImage(properties=['predicted'], reducer=ee.Reducer.first())

# Increase point size using focal_max()
test_image = test_image.focal_max(radius=10, units='meters')

vis = {
  'min': 0,
  'max': 3,
  'palette': ['00FF00', '00FFFF', 'FFFF00', 'FF0000'] 
}

# AOI boundary
AOI = ee.FeatureCollection('projects/svevazanettieo1/assets/Dubai/Area_of_interest_Dubai')

# Create a map centered on AOI
Map = geemap.Map(center=[25.276987, 55.296249], zoom=10)
Map.addLayer(AOI, {'color': 'gray'}, 'AOI')
Map.addLayer(sentinel2018, vis_params, 'Sentinel-2 2018')

# Add the predictor layers
Map.addLayer(test_image, vis,'Prediction')
Map

# Testing 2024

In [ ]:
X_shuffled_test_2024, y_shuffled_test_2024, testing_points_df_2024 = shuffle(X_test_2024, y_test_2024, testing_points_df_2024, random_state=42)
y_shuffled_test_2024 = np.argmax(y_shuffled_test_2024, axis=1)

# Predicting using the model
y_pred_test_2024 = clf.predict(X_shuffled_test_2024)  # Get predictions from the model

In [ ]:
# Compute confusion matrix
cm_2024 = confusion_matrix(y_shuffled_test_2024, y_pred_test_2024)

# Plot
fig, ax = plt.subplots(figsize=(8, 6)) 
disp = ConfusionMatrixDisplay(confusion_matrix=cm_2024, display_labels=['vegetation','water','soil','urban'])
disp.plot(cmap="Blues", values_format="d", ax=ax, colorbar=False)  # use ax
ax.set_title("Confusion Matrix", fontsize=16)
plt.tight_layout()
plt.show()

In [ ]:
testing_points_df_2024['predicted'] = y_pred_test_2024

testing_points_df_2024['geometry'] = testing_points_df_2024.apply(make_geometry, axis=1)
gdf_2024 = gpd.GeoDataFrame(testing_points_df_2024, geometry='geometry')
gdf_2024.set_crs(epsg=4326, inplace=True)

In [ ]:
# Select only the necessary columns for visualization
test_points_RF = geemap.geopandas_to_ee(gdf_2024[['predicted', 'geometry']])

# Convert FeatureCollection to Image
test_image = test_points_RF.reduceToImage(properties=['predicted'], reducer=ee.Reducer.first())

# Increase point size using focal_max()
test_image = test_image.focal_max(radius=10, units='meters')

# Create a map centered on AOI
Map = geemap.Map(center=[25.276987, 55.296249], zoom=10)
Map.addLayer(AOI, {'color': 'gray'}, 'AOI')
Map.addLayer(sentinel2018, vis_params, 'Sentinel-2 2018')

# Add the predictor layers
Map.addLayer(test_image, vis,'Prediction')
Map